SPDX-FileCopyrightText: Copyright 2025-2026 Arm Limited and/or its affiliates <open-source-office@arm.com>\
SPDX-License-Identifier: Apache-2.0

# Model Fine-Tuning Example
---

This notebook shows how to use the Neural Graphics Model Gym to run fine-tuning of the Neural Super Sampling model.

### Prerequisites
Familiarity with the concepts in
- [model training tutorial](model_training_example.ipynb)
- [model evaluation tutorial](model_evaluation_example.ipynb)

### Environment Setup
**Before running the notebook:**

Check the environment prerequisites and follow the set up instructions in the [README.md](../README.md)

In [ ]:
# Import the ng_model_gym package.
import ng_model_gym as ngmg

### Inspect Configuration Parameters

In [ ]:
import logging
from pathlib import Path

# Load the fine-tuning config (some parameters are set differently)
cfg_path = Path("./assets/nss/finetune_config.json")
config = ngmg.load_config_file(cfg_path)

# Enable minimal logging for ng_model_gym.
# For more detailed logs set log_level to logging.INFO or logging.DEBUG.
ngmg.logging_config(config, log_level=logging.ERROR)

In [ ]:
from rich import print as rprint

# Let's inspect the loaded config object.
rprint(config)

### Run Fine-Tuning

Fine-tuning can be done by using the do_training function as before. This time we pass a path to a previously trained weights file. These weights act as the starting point from which we can fine-tune.

In [ ]:
from ng_model_gym import TrainEvalMode

# Update the config to run validation at the end of each epoch to estimate the image quality
config.train.perform_validate = False

# To fine-tune, we pass in the Path to a previously trained .pt model file.
# For this demo, we will use the provided pretrained .pt file
best_ckpt_path = ngmg.do_training(
    config,
    training_mode=TrainEvalMode.FP32,
    finetune_model_path="./data/nss/weights/nss_v0.1.0_fp32.pt",
)

### Compare Evaluation Metrics

Now that we have a model which is fine-tuned to our specific training sequence, we can compare its evaluation metrics with those of the pretrained weights to confirm that the fine-tuned model performs better on this scene.

First, get the evaluation metrics for the pretrained weights:

In [ ]:
ngmg.do_evaluate(
    config, Path("./data/nss/weights/nss_v0.1.0_fp32.pt"), TrainEvalMode.FP32
)

And now compare the final checkpoint from fine-tuning with the evaluation results for the pretrained weights:

In [ ]:
ngmg.do_evaluate(config, best_ckpt_path, TrainEvalMode.FP32)

For even better results, we can increase `config.dataset.recurrent_samples` to 16 before running fine-tuning. 

Note: this will significantly increase the execution time of the do_training function.